<a href="https://colab.research.google.com/github/CargofirstQaho/Commodity_analysis_GC/blob/main/wheatFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip
!pip install --only-binary=:all: opencv-python-headless==4.8.0.76 numpy==1.24.4 pillow==9.5.0 scikit-image==0.21.0
!pip install --only-binary=:all: streamlit==1.38.0 pyngrok==7.2.0 watchdog==4.0.0 requests

import os
# set your token here then delete/clear this cell after use
os.environ["NGROK_AUTHTOKEN"] = "35gSLvFYhBHyiL1IDMvwLuuGCxK_3wDSUYrMugCvGS8MgPSFk"

app_code = r"""
# app.py
import streamlit as st
import numpy as np
import cv2
from PIL import Image
from io import BytesIO

st.set_page_config(page_title='Wheat Quality Analyzer', layout='wide')
st.title('Wheat Quality Analyzer')

st.markdown('Counts: total, good, broken, foreign matter, spoiled, discoloured. Color classes: light brown, brown, dark brown, black. Averages reported in mm. Percentages show class share of total detected grains.')

# Sidebar tuning (wheat-specific)
st.sidebar.header('Tunable thresholds')
broken_ratio = st.sidebar.slider('Broken length ratio (of median)', 0.4, 0.95, 0.70, 0.01)
min_area = st.sidebar.slider('Min contour area (px) filter', 20, 400, 80, 1)

# Foreign matter heuristics
foreign_solidity_thresh = st.sidebar.slider('Foreign solidity threshold', 0.10, 0.95, 0.60, 0.01)
foreign_area_multiplier = st.sidebar.slider('Foreign area multiplier (vs median area)', 1.0, 10.0, 5.0, 0.1)
foreign_aspect_ratio = st.sidebar.slider('Foreign extreme aspect ratio', 1.5, 6.0, 3.5, 0.1)
foreign_circularity_thresh = st.sidebar.slider('Foreign circularity threshold (lower = more foreign)', 0.10, 1.00, 0.35, 0.01)

# Discolour/spoiled heuristics (LAB space)
discolour_percentile = st.sidebar.slider('Discolour b-channel percentile (lower b = greyer)', 1, 50, 25, 1)
spoiled_L_percentile = st.sidebar.slider('Spoiled darkness L percentile (lower L = darker)', 1, 40, 15, 1)

# Color labeling
k_colors = st.sidebar.slider('Color clusters (k-means on LAB)', 2, 5, 4, 1)
light_dark_split = st.sidebar.slider('Light/Dark L threshold (0-255)', 60, 190, 120, 1)

# mm calibration (replace if you have known mm-per-pixel)
target_avg_mm = st.sidebar.slider('Target avg kernel length for calibration (mm)', 5.5, 8.5, 7.0, 0.1)

# Image processing utils
def clahe_rgb(img_bgr):
    lab = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2LAB)
    L, A, B = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    Lc = clahe.apply(L)
    labc = cv2.merge([Lc, A, B])
    return cv2.cvtColor(labc, cv2.COLOR_LAB2BGR)

def preprocess(img):
    # Contrast-normalize, then grayscale threshold for wheat kernels
    img_c = clahe_rgb(img)
    gray = cv2.cvtColor(img_c, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY_INV, 51, 5)
    kernel = np.ones((3,3), np.uint8)
    opened = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)
    cleaned = cv2.morphologyEx(opened, cv2.MORPH_CLOSE, kernel, iterations=1)
    dilated = cv2.dilate(cleaned, kernel, iterations=1)
    return img_c, gray, dilated

def extract_grain_contours(binary_mask, min_area_px):
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    filtered = [c for c in contours if cv2.contourArea(c) > float(min_area_px)]
    return filtered

def circularity(contour):
    area = cv2.contourArea(contour)
    perim = cv2.arcLength(contour, True) + 1e-6
    return float(4.0 * np.pi * area / (perim * perim))

def grain_features(img_bgr, gray, contour):
    area = cv2.contourArea(contour)
    rect = cv2.minAreaRect(contour)
    (cx, cy), (w, h), angle = rect
    length = max(w, h)
    width = min(w, h)
    aspect_ratio = (length + 1.0) / (width + 1.0)

    # masks
    mask = np.zeros_like(gray)
    cv2.drawContours(mask, [contour], -1, 255, -1)

    mean_intensity = float(cv2.mean(gray, mask=mask)[0])

    hull = cv2.convexHull(contour)
    hull_area = cv2.contourArea(hull) + 1e-6
    solidity = float(area / hull_area)
    circ = circularity(contour)

    # LAB features
    lab = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2LAB)
    L = lab[:,:,0]; A = lab[:,:,1]; B = lab[:,:,2]
    mean_L = float(cv2.mean(L, mask=mask)[0])
    mean_A = float(cv2.mean(A, mask=mask)[0])
    mean_B = float(cv2.mean(B, mask=mask)[0])

    polygon = contour.reshape(-1, 2).tolist()

    return {
        "contour": contour,
        "polygon": polygon,
        "area": float(area),
        "length_px": float(length),
        "width_px": float(width),
        "aspect_ratio": float(aspect_ratio),
        "mean_intensity": mean_intensity,
        "solidity": solidity,
        "circularity": circ,
        "cx": float(cx),
        "cy": float(cy),
        "L": mean_L,
        "A": mean_A,
        "B": mean_B
    }

def classify_wheat(features_list,
                   broken_ratio,
                   discolour_percentile,
                   spoiled_L_percentile,
                   foreign_solidity_thresh,
                   foreign_area_multiplier,
                   foreign_ar_thresh,
                   foreign_circ_thresh):
    results = []
    # robust medians for scale references
    lengths = np.array([f["length_px"] for f in features_list]) if features_list else np.array([0.0])
    areas = np.array([f["area"] for f in features_list]) if features_list else np.array([0.0])
    Ls = np.array([f["L"] for f in features_list]) if features_list else np.array([0.0])
    Bs = np.array([f["B"] for f in features_list]) if features_list else np.array([0.0])

    median_len = float(np.median(lengths)) if len(lengths) else 0.0
    broken_threshold = broken_ratio * median_len

    # lower B (bluer/greyer) → more discolour; lower L → darker
    discolour_threshold = float(np.percentile(Bs, discolour_percentile)) if len(Bs) else 0.0
    spoiled_threshold = float(np.percentile(Ls, spoiled_L_percentile)) if len(Ls) else 0.0

    median_area = float(np.median(areas)) if len(areas) else 0.0

    for f in features_list:
        is_broken = f["length_px"] < broken_threshold

        is_foreign = (f["solidity"] < foreign_solidity_thresh) or \
                     (f["area"] > foreign_area_multiplier * median_area) or \
                     (f["aspect_ratio"] > foreign_ar_thresh) or \
                     (f["circularity"] < foreign_circ_thresh)

        is_spoiled = (f["L"] <= spoiled_threshold) and (f["solidity"] < 0.8)
        is_discoloured = (f["B"] <= discolour_threshold) and (not is_spoiled)

        is_good = (not is_broken) and (not is_foreign) and (not is_spoiled) and (not is_discoloured)

        label = "good"
        if is_foreign:
            label = "foreign"
        elif is_broken:
            label = "broken"
        elif is_spoiled:
            label = "spoiled"
        elif is_discoloured:
            label = "discoloured"

        results.append({**f,
                        "is_broken": bool(is_broken),
                        "is_foreign": bool(is_foreign),
                        "is_spoiled": bool(is_spoiled),
                        "is_discoloured": bool(is_discoloured),
                        "is_good": bool(is_good),
                        "label": label})

    meta = {
        "median_len_px": median_len,
        "broken_threshold_px": broken_threshold,
        "discolour_threshold_B": discolour_threshold,
        "spoiled_threshold_L": spoiled_threshold,
        "median_area_px": median_area
    }
    return results, meta

def color_label_from_lab(L, A, B, light_dark_split):
    # Simple interpretable bins
    # Light vs dark by L; chroma by A/B roughly maps to brownness
    if L <= 40:
        return "black"
    if L <= light_dark_split:
        # darker range
        if B >= 140 and A >= 140:
            return "dark brown"
        else:
            return "brown"
    else:
        # lighter range
        if B >= 140 and A >= 140:
            return "brown"
        else:
            return "light brown"

def annotate_polygons(img, results):
    annotated = img.copy()
    colors = {
        "good": (0, 200, 0),
        "broken": (0, 0, 255),
        "discoloured": (0, 165, 255),
        "spoiled": (128, 0, 128),
        "foreign": (255, 0, 0)
    }
    for r in results:
        contour = r["contour"]
        label = r.get("label", "good")
        color = colors.get(label, (255,255,255))
        cv2.drawContours(annotated, [contour], -1, color, 2, cv2.LINE_AA)
        cx, cy = int(r["cx"]), int(r["cy"])
        cv2.circle(annotated, (cx, cy), 2, color, -1)
    return annotated

def pixels_to_mm(avg_pixel_length_px, target_avg_mm=7.0):
    if avg_pixel_length_px <= 0:
        return 0.0
    return float(target_avg_mm / avg_pixel_length_px)

# Streamlit inputs
uploaded = st.file_uploader("Upload a wheat image (jpg/png)", type=["jpg","jpeg","png"])
camera_img = st.camera_input("Or capture from your camera")

image_bytes = None
if uploaded is not None:
    image_bytes = uploaded.read()
elif camera_img is not None:
    image_bytes = camera_img.getvalue()

if image_bytes is not None:
    pil = Image.open(BytesIO(image_bytes)).convert("RGB")
    img = np.array(pil)[:, :, ::-1].copy()
    img_c, gray, binary = preprocess(img)
    contours = extract_grain_contours(binary, min_area)
    feats = [grain_features(img_c, gray, c) for c in contours]
    results, meta = classify_wheat(
        feats,
        broken_ratio,
        discolour_percentile,
        spoiled_L_percentile,
        foreign_solidity_thresh,
        foreign_area_multiplier,
        foreign_aspect_ratio,
        foreign_circularity_thresh
    )

    # Color labels
    for r in results:
        r["color_label"] = color_label_from_lab(r["L"], r["A"], r["B"], light_dark_split)

    total = len(results)
    count_good = sum(1 for r in results if r["label"] == "good")
    count_broken = sum(1 for r in results if r["label"] == "broken")
    count_discoloured = sum(1 for r in results if r["label"] == "discoloured")
    count_spoiled = sum(1 for r in results if r["label"] == "spoiled")
    count_foreign = sum(1 for r in results if r["label"] == "foreign")

    # Color counts
    color_classes = ["light brown", "brown", "dark brown", "black"]
    color_counts = {c: sum(1 for r in results if r.get("color_label")==c) for c in color_classes}

    avg_len_px = float(np.mean([r["length_px"] for r in results])) if total else 0.0
    avg_wid_px = float(np.mean([r["width_px"] for r in results])) if total else 0.0
    mm_per_px = pixels_to_mm(avg_len_px, target_avg_mm=target_avg_mm) if avg_len_px > 0 else 0.0
    avg_len_mm = avg_len_px * mm_per_px
    avg_wid_mm = avg_wid_px * mm_per_px

    pct_good = (count_good / total * 100.0) if total else 0.0
    pct_broken = (count_broken / total * 100.0) if total else 0.0
    pct_discoloured = (count_discoloured / total * 100.0) if total else 0.0
    pct_spoiled = (count_spoiled / total * 100.0) if total else 0.0
    pct_foreign = (count_foreign / total * 100.0) if total else 0.0

    col1, col2 = st.columns(2)
    with col1:
        st.subheader("Counts")
        st.metric("Total detected", total)
        st.metric("Good (normal)", f"{count_good} ({pct_good:.1f}%)")
        st.metric("Broken", f"{count_broken} ({pct_broken:.1f}%)")
        st.metric("Discoloured", f"{count_discoloured} ({pct_discoloured:.1f}%)")
        st.metric("Spoiled", f"{count_spoiled} ({pct_spoiled:.1f}%)")
        st.metric("Foreign matter", f"{count_foreign} ({pct_foreign:.1f}%)")

        st.subheader("Averages (mm)")
        st.write(f"Average length: **{avg_len_mm:.2f} mm**")
        st.write(f"Average width: **{avg_wid_mm:.2f} mm**")
        st.caption("Calibration: detected avg kernel length mapped to the sidebar value (default 7.0 mm). Replace with known mm-per-pixel for accurate absolute sizing.")

        st.subheader("Color distribution")
        for k in color_classes:
            pct = (color_counts[k] / total * 100.0) if total else 0.0
            st.write(f"- **{k}:** {color_counts[k]} ({pct:.1f}%)")

    with col2:
        st.subheader("Annotated (polygons)")
        annotated = annotate_polygons(img_c, results)
        display_img = annotated[:, :, ::-1]
        st.image(display_img, caption="Green=Good, Red=Broken, Orange=Discoloured, Purple=Spoiled, Blue=Foreign", use_column_width=True)

        buffer = BytesIO()
        pil_out = Image.fromarray(display_img)
        pil_out.save(buffer, format="PNG")
        buffer.seek(0)
        st.download_button("Download annotated image", data=buffer, file_name="annotated_wheat.png", mime="image/png")

else:
    st.info("Upload an image or use the camera to start analysis.")
"""
with open("app.py", "w") as f:
    f.write(app_code)
print("app.py written")

from google.colab import files
uploaded = files.upload()

import subprocess, time, os, requests
from pyngrok import ngrok

token = os.environ.get("NGROK_AUTHTOKEN", "")
if token:
    ngrok.set_auth_token(token)

proc = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", "8501", "--server.headless", "true"],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE
)

start = time.time()
timeout = 40
while True:
    try:
        r = requests.get("http://localhost:8501")
        if r.status_code < 500:
            break
    except:
        pass
    if time.time() - start > timeout:
        print("Warning: Streamlit did not respond within timeout; check logs.")
        break
    time.sleep(0.5)

public_url = ngrok.connect(8501, "http").public_url
print("Public URL:", public_url)
print("If the URL fails, restart this cell. Remove ngrok token cell before sharing notebook.")

import time, sys
for _ in range(200):
    line = proc.stderr.readline()
    if not line:
        time.sleep(0.1)
        continue
    try:
        print(line.decode('utf-8', errors='ignore').rstrip())
    except:
        print(line)


  Using cached opencv_python_headless-4.8.0.76-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
ERROR: Could not find a version that satisfies the requirement numpy==1.24.4 (from versions: 1.26.0, 1.26.1, 1.26.2, 1.26.3, 1.26.4, 2.0.0, 2.0.1, 2.0.2, 2.1.0, 2.1.1, 2.1.2, 2.1.3, 2.2.0, 2.2.1, 2.2.2, 2.2.3, 2.2.4, 2.2.5, 2.2.6, 2.3.0, 2.3.1, 2.3.2, 2.3.3, 2.3.4, 2.3.5)
ERROR: No matching distribution found for numpy==1.24.4
app.py written


Saving Gemini_Generated_Image_wov1y8wov1y8wov1.png to Gemini_Generated_Image_wov1y8wov1y8wov1.png
Public URL: https://camphoric-nonevanescently-maynard.ngrok-free.dev
If the URL fails, restart this cell. Remove ngrok token cell before sharing notebook.


2025-11-26 12:06:21.392 MediaFileHandler: Missing file 5788d59cad97efcad4797c90483f7fd02166f61ab593bb8eb3d1d3c2.jpg
